In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

In [6]:
df=pd.read_csv('^NSEBANK.csv')
df.rename(columns={'Date ':'Date','Open ':'Open','High ':'High','Low ':'Low','Close ':'Close'},inplace=True)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close'], dtype='object')

In [24]:
#Swing Lows
support=df
support=round(support,2)
support.set_index('Date',inplace=True)
support.index=pd.to_datetime(support.index,format="%d/%m/%y").date
support=support[support.Close==support.Close.rolling(5,center=True).min()]
support['Swing']='SL'
#support['Swing']=np.where(support.Low<support.Low.shift(1),'LL','HL')
support.tail()

,Open,High,Low,Close,Swing
2023-11-01,42694.25,42815.15,42589.65,42700.95,SL
2023-11-17,43656.25,43872.85,43513.85,43583.95,SL
2023-11-22,43663.60,43692.50,43230.95,43449.60,SL
2023-12-13,47190.75,47201.95,46886.90,47092.25,SL
2023-12-20,48085.35,48166.15,47202.65,47445.30,SL


In [25]:
#Swing Highs
resistance=df
resistance=round(resistance,2)
resistance.set_index('Date',inplace=True)
resistance.index=pd.to_datetime(resistance.index,format="%d/%m/%y").date
resistance=resistance[resistance.Close==resistance.Close.rolling(5,center=True).max()]
resistance['Swing']='SH'
#support['Swing']=np.where(support.Low<support.Low.shift(1),'LL','HL')
resistance.tail()

,Open,High,Low,Close,Swing
2023-11-21,43790.95,43790.95,43586.45,43689.15,SH
2023-12-05,46895.90,47230.55,46653.65,47012.25,SH
2023-12-11,47233.30,47588.00,47186.60,47314.25,SH
2023-12-15,47939.90,48219.95,47632.85,48143.55,SH
2023-12-28,48479.05,48636.45,48343.15,48508.55,SH


In [26]:
temp=pd.concat([support,resistance])
temp

,Open,High,Low,Close,Swing
2022-01-24,37601.95,37694.70,36375.35,36947.55,SL
2022-01-28,38246.60,38421.70,37581.50,37689.40,SL
2022-02-07,38592.05,38837.55,37802.70,37995.45,SL
2022-02-14,37664.00,37796.55,36828.05,36908.55,SL
2022-02-17,38042.10,38124.50,37425.60,37531.65,SL
...,...,...,...,...,...
2023-11-21,43790.95,43790.95,43586.45,43689.15,SH
2023-12-05,46895.90,47230.55,46653.65,47012.25,SH
2023-12-11,47233.30,47588.00,47186.60,47314.25,SH
2023-12-15,47939.90,48219.95,47632.85,48143.55,SH


In [27]:
dt=df
dt.set_index('Date',inplace=True)
dt.index=pd.to_datetime(dt.index,format="%d/%m/%y").date
dt=round(dt,2)

In [28]:
dt = dt.merge(temp[['Swing']], left_index=True, right_index=True, how='left')

In [30]:
#dt=dt.drop(['Adj Close','Volume'],axis=1)

In [31]:
dt.tail()

,Open,High,Low,Close,Swing
2023-12-26,47576.40,47838.45,47411.65,47724.85,NaN
2023-12-27,47818.50,48347.65,47806.70,48282.20,NaN
2023-12-28,48479.05,48636.45,48343.15,48508.55,SH
2023-12-29,48374.85,48477.70,48091.85,48292.25,NaN
2024-01-01,48203.45,48450.00,48044.15,48234.30,NaN


In [32]:
#Stop Loss
def long_stop(long_price,system_low):
    system_points=long_price - system_low
    
    if  long_price*0.02 < system_points:
        stop_price=round(long_price*0.98,2)
    else:
        stop_price=system_low
    
    return stop_price

def short_stop(short_price,system_high):
    system_points=system_high - short_price
    
    if short_price*0.02 < system_points:
        stop_price=round(short_price*1.02,2)
    else:
        stop_price=system_high 
    
    return stop_price

In [33]:
long_price=0
long_date=0
long_entry_price=0
long_stop_price=0
system_low=0
long_position=0
long_exit_date=0
long_exit_price=0
lp=0                            #Copy of long_price
sl=0                            #Copy of system_low
SL_cnt=0 

long_trades=[]
long_trail=[]

short_price=0
short_date=0
short_entry_price=0
short_stop_price=0
system_high=0
short_position=0
short_exit_date=0
short_exit_price=0
sp=0                            #Copy of short_price
sh=0                            #Copy of system_high
SH_cnt=0  

short_trades=[]
short_trail=[]

long_position=None
short_position=None

for i in range(len(dt)):
    
    #Long Entry Point
    if dt.Swing.iloc[i] == 'SH':
        long_price = dt.Close.iloc[i]
        system_low = dt.Close.iloc[i]
        
    #System Low    
    if dt.Swing.iloc[i] != 'SH' and long_price != 0:
        
        if dt.Close.iloc[i] < system_low:
            system_low=dt.Close.iloc[i]
            
    if dt.Swing.iloc[i] == 'SL' and dt.Close.iloc[i] > system_low:
        system_low=dt.Close.iloc[i]
    
    #Short Entry Point
    if dt.Swing.iloc[i] == 'SL':
        short_price = dt.Close.iloc[i]
        system_high = dt.Close.iloc[i]
        #print(short_price)
        #print(i)
    
    #System High 
    if dt.Swing.iloc[i] != 'SL' and short_price != 0:
        
        if dt.Close.iloc[i] > system_high:
            system_high = dt.Close.iloc[i]
            #print(system_high)
            
    if dt.Swing.iloc[i] == 'SH' and dt.Close.iloc[i] < system_high:
        system_high=dt.Close.iloc[i]
            
    #Long Entry Trigger
    if dt.Swing.iloc[i] != 'SH' and long_price!=0 and long_position==None:
        
        if dt.Open.iloc[i] > long_price and dt.Close.iloc[i] > long_price:
            long_entry_price=dt.Open.iloc[i]
            #print(long_entry_price)
            long_date=dt.index[i]
            #print(long_date)
            long_stop_price=long_stop(long_price,system_low)
            sl=system_low
            lp=long_price
            long_position=True
        elif dt.Close.iloc[i] > long_price:
            long_entry_price=dt.Close.iloc[i]
            #print(long_entry_price)
            long_date=dt.index[i]
            #print(long_date)
            long_stop_price=long_stop(long_price,system_low)
            sl=system_low
            lp=long_price
            long_position=True
        
    #Long Stops
    if long_position == True:
        if dt.Open.iloc[i] <= sl*0.98:
            long_exit_date=dt.index[i]
            long_exit_price=dt.Open.iloc[i]
        elif dt.Close.iloc[i] <= long_stop_price:
            long_exit_date=dt.index[i]
            long_exit_price=long_stop_price
        elif dt.Swing.iloc[i] == 'SL':
            long_stop_price=dt.Close.iloc[i]
            SL_cnt=SL_cnt+1
        elif dt.Close.iloc[i] >= lp*1.02 and SL_cnt==0:
            long_stop_price=long_entry_price
            
        #Exiting the current long trade just for the sake of recording    
        if i==len(dt)-2:
            long_exit_date=dt.index[i]
            long_exit_price=dt.Close.iloc[i]
            
        long_trail.append({'Date':dt.index[i],'Long Stop':long_stop_price})
    
        
    #Long Trade Records    
    if long_exit_price !=0:
        long_trades.append({'Trade Direction':'Long',
                        'Entry Date':long_date,
                       'Entry Price':long_entry_price,
                       'Stop': long_stop_price,
                      'Exit Price': long_exit_price,
                       'Exit Date': long_exit_date,
                      'Profit': long_exit_price - long_entry_price})    
        
        long_exit_price=0
        long_date=0
        long_entry_price=0
        long_stop_price=0
        long_exit_date=0
        long_position=None
        SL_cnt=0      
    
    #Short Entry Trigger
    if dt.Swing.iloc[i] != 'SL' and short_price != 0 and short_position == None:
              
        if dt.Open.iloc[i] < short_price and dt.Close.iloc[i] < short_price:
            short_entry_price=dt.Open.iloc[i]
            short_date=dt.index[i]
            short_stop_price=short_stop(short_price,system_high)
            sh=system_high
            sp=short_price
            short_position=True
        elif dt.Close.iloc[i] < short_price:
            short_entry_price=dt.Close.iloc[i]
            short_date = dt.index[i]
            short_stop_price=short_stop(short_price,system_high)
            sh=system_high
            sp=short_price
            short_position=True
            
    #Short Stops
    if short_position == True:
        if dt.Open.iloc[i] >= sh*1.02:
            short_exit_date=dt.index[i]
            short_exit_price=dt.Open.iloc[i]
        elif dt.Close.iloc[i] >=  short_stop_price:
            short_exit_date=dt.index[i]
            short_exit_price=short_stop_price
        elif dt.Swing.iloc[i] == 'SH':
            short_stop_price = dt.Close.iloc[i]
            SH_cnt+=1
        elif dt.Close.iloc[i] <= sp*0.98 and SH_cnt==0:
            short_stop_price=short_entry_price
                   
        short_trail.append({'Date':dt.index[i],'Short Stop':short_stop_price})  
        
    #Short Trade Records 
    if short_exit_price !=0:
        short_trades.append({'Trade Direction':'Short',
                        'Entry Date':short_date,
                       'Entry Price':short_entry_price,
                       'Stop': short_stop_price,
                      'Exit Price': short_exit_price,
                      'Exit Date': short_exit_date,
                      'Profit': short_entry_price - short_exit_price})    
        
        short_exit_price=0
        short_date=0
        short_entry_price=0
        short_stop_price=0
        short_exit_date=0
        short_position=None
        SH_cnt=0

In [34]:
short_trades_df=pd.DataFrame(short_trades)
short_trail_df = pd.DataFrame(short_trail) 

short_trail_df.set_index('Date', inplace=True)
short_trades_df.set_index('Entry Date', inplace=True)

In [35]:
short_trades_df

,Trade Direction,Entry Price,Stop,Exit Price,Exit Date,Profit
Entry Date,,,,,,
2022-02-22,Short,36833.00,36428.55,36428.55,2022-04-01,404.45
2022-04-13,Short,37463.40,34301.90,34301.90,2022-05-25,3161.50
2022-06-03,Short,35275.05,33191.75,33191.75,2022-06-24,2083.30
2022-09-22,Short,40630.60,39282.85,39282.85,2022-10-14,1347.75
2022-12-21,Short,42617.95,42617.95,42617.95,2022-12-26,0.00
2023-01-25,Short,41647.65,41499.70,41499.70,2023-02-08,147.95
2023-02-17,Short,41131.75,41131.75,41131.75,2023-03-03,0.00
2023-03-13,Short,39564.70,39564.70,39564.70,2023-03-17,0.00
2023-08-02,Short,45234.65,44495.25,44495.25,2023-09-04,739.40


In [36]:
#short_trail_df

In [37]:
long_trades_df=pd.DataFrame(long_trades)
long_trail_df = pd.DataFrame(long_trail) 

long_trail_df.set_index('Date', inplace=True)
long_trades_df.set_index('Entry Date', inplace=True)

In [38]:
long_trades_df

,Trade Direction,Entry Price,Stop,Exit Price,Exit Date,Profit
Entry Date,,,,,,
2022-02-01,Long,38460.45,38460.45,38460.45,2022-02-07,0.00
2022-04-01,Long,37148.50,37557.35,37557.35,2022-04-13,408.85
2022-05-25,Long,34491.80,35487.40,35487.40,2022-06-03,995.60
2022-06-24,Long,33434.80,38297.75,38297.75,2022-08-29,4862.95
2022-09-08,Long,40208.95,40776.80,40776.80,2022-09-22,567.85
2022-10-14,Long,39446.05,43103.75,43103.75,2022-12-07,3657.70
2022-12-08,Long,43596.85,43219.50,43219.50,2022-12-21,-377.35
2023-01-18,Long,42458.00,42167.55,42167.55,2023-01-25,-290.45
2023-02-08,Long,41542.05,41374.65,41374.65,2023-02-13,-167.40


In [39]:
#long_trail_df

In [40]:
dt = dt.merge(short_trail_df[['Short Stop']], left_index=True, right_index=True, how='left') \
          .merge(short_trades_df[['Entry Price']], left_index=True, right_index=True, how='left')

In [41]:
dt = dt.merge(long_trail_df[['Long Stop']], left_index=True, right_index=True, how='left') \
          .merge(long_trades_df[['Entry Price']], left_index=True, right_index=True, how='left')

In [78]:
bt=pd.merge(short_trades_df.reset_index(),long_trades_df.reset_index(),how='outer')
bt=bt.set_index('Entry Date')
bt=bt.sort_index(axis=0)
bt.tail()

,Trade Direction,Entry Price,Stop,Exit Price,Exit Date,Profit
Entry Date,,,,,,
2023-09-27,Short,44465.90,44360.60,44360.60,2023-10-11,105.30
2023-10-11,Long,44554.90,43886.50,43886.50,2023-10-19,-668.40
2023-10-19,Short,43616.75,43039.15,43039.15,2023-11-03,577.60
2023-11-03,Long,43318.30,43583.95,43583.95,2023-11-22,265.65
2023-11-24,Long,43769.10,47445.30,48292.25,2023-12-29,4523.15


In [79]:
#Multiplying the Points gained/Lost with Bank Nifty Futures Lot Size, 
#Premium aka Cost of Carry has not been considered!!!
bt['Profit']=bt['Profit'].apply(lambda x:x*15) 

In [80]:
bt.tail()


,Trade Direction,Entry Price,Stop,Exit Price,Exit Date,Profit
Entry Date,,,,,,
2023-09-27,Short,44465.90,44360.60,44360.60,2023-10-11,1579.50
2023-10-11,Long,44554.90,43886.50,43886.50,2023-10-19,-10026.00
2023-10-19,Short,43616.75,43039.15,43039.15,2023-11-03,8664.00
2023-11-03,Long,43318.30,43583.95,43583.95,2023-11-22,3984.75
2023-11-24,Long,43769.10,47445.30,48292.25,2023-12-29,67847.25


In [81]:
bt['DD']=np.where(bt.Profit.iloc[0]<0,bt.Profit.iloc[0],0)
for i in range(1,len(bt)):
    if bt.Profit.iloc[i] + bt.DD.iloc[i-1] < 0:
        bt.DD.iloc[i]=bt.Profit.iloc[i] + bt.DD.iloc[i-1]
    else:
        bt.DD.iloc[i]=0
bt.head()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Trade Direction,Entry Price,Stop,Exit Price,Exit Date,Profit,DD
Entry Date,,,,,,,
2022-02-01,Long,38460.45,38460.45,38460.45,2022-02-07,0.00,0.0
2022-02-22,Short,36833.00,36428.55,36428.55,2022-04-01,6066.75,0.0
2022-04-01,Long,37148.50,37557.35,37557.35,2022-04-13,6132.75,0.0
2022-04-13,Short,37463.40,34301.90,34301.90,2022-05-25,47422.50,0.0
2022-05-25,Long,34491.80,35487.40,35487.40,2022-06-03,14934.00,0.0


In [82]:
bt.loc['Cumulative Profit'] = pd.Series(bt['Profit'].sum(), index=['Profit'])
#bt.loc['Max DD'] = pd.Series(bt['DD'].Max(), index=['DD'])
print(bt)


                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [83]:
bt.loc['Max DD'] = pd.Series(bt['DD'].min(), index=['DD'])
print(bt)

                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [84]:
bt.loc['Margin']=pd.Series(100000,index=['Profit'])
print(bt)

                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [85]:
bt.loc['Profit Per Anum']=pd.Series(bt.loc['Cumulative Profit','Profit']/2,index=['Profit'])
print(bt)

                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [87]:
bt.loc['Return Perc']=pd.Series((bt.loc['Profit Per Anum','Profit']/bt.loc['Margin','Profit'])*100,index=['Profit'])
print(bt)

                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [89]:
bt.loc['Calmer']=pd.Series(bt.loc['Profit Per Anum','Profit']/abs(bt.loc['Max DD','DD']),index=['Profit'])
print(bt)

                  Trade Direction  Entry Price      Stop  Exit Price  \
Entry Date                                                             
2022-02-01                   Long     38460.45  38460.45    38460.45   
2022-02-22                  Short     36833.00  36428.55    36428.55   
2022-04-01                   Long     37148.50  37557.35    37557.35   
2022-04-13                  Short     37463.40  34301.90    34301.90   
2022-05-25                   Long     34491.80  35487.40    35487.40   
2022-06-03                  Short     35275.05  33191.75    33191.75   
2022-06-24                   Long     33434.80  38297.75    38297.75   
2022-09-08                   Long     40208.95  40776.80    40776.80   
2022-09-22                  Short     40630.60  39282.85    39282.85   
2022-10-14                   Long     39446.05  43103.75    43103.75   
2022-12-08                   Long     43596.85  43219.50    43219.50   
2022-12-21                  Short     42617.95  42617.95    4261

In [90]:
bt.to_csv('Backtest_Results.csv')